# 01 Backwards and Forwards Models

The final bit for today is looking at ensembling backwards and forwards models. 

The technique can be boiled down as simply reversing the entire sentence. We train two models, one forwards and one backwards, and ensemble the two of them. Both the language model and the downstream'd model have this augmentation applied.

We need to make two modifications to get this to work, a transform to reverse and a modified `learner` to enable us to get the pretrained langauge model:

In [ ]:
!pip install fastai2

In [ ]:
from fastai2.text.all import *

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)

In [ ]:
df = pd.read_csv(path/'texts.csv')

We reverse each individual text inside of the batch:

In [ ]:
def reverse_text(x): return torch.stack([a.flip(0) for a in x])

Let's build two LM DataLoaders to see this in action, one with reverse and one without:

First the regular (forwards) `DataBlock` and `DataLoaders`:

In [ ]:
lm_block = TextBlock.from_df('text', is_lm=True, res_col_name='tok_text')
dblock = DataBlock(blocks=lm_block,
                   get_x=ColReader('tok_text'),
                   splitter=RandomSplitter(0.1, seed=42))

In [ ]:
dls = dblock.dataloaders(df, bs=64, seq_len=72, num_workers=4)

Then our new backwards one, with an added `batch_tfm` of our `reverse_text` function:

In [ ]:
lm_block = TextBlock.from_df('text', is_lm=True, res_col_name='tok_text')
dblock = DataBlock(blocks=lm_block,
                   get_x=ColReader('tok_text'),
                   splitter=RandomSplitter(0.1, seed=42),
                   batch_tfms=reverse_text)

In [ ]:
dls_r = dblock.dataloaders(df, bs=64, seq_len=72, num_workers=4)

Let's take a look at our backwards `DataLoader` to make sure it is working:

In [ ]:
dls_r.show_batch()

,text,text_
0,make go xxmaj \n\n ! you tell will i \n\n ? do to going you are what now so xxmaj ? stacking needs xxunk the and telly the on football no is there and night stormy and dark a 's it xxmaj . home at are though you xxmaj ? law in mother the visiting is wife the xxmaj ? over - sleep a on away are xxunk your so xxmaj xxbos,an make go xxmaj \n\n ! you tell will i \n\n ? do to going you are what now so xxmaj ? stacking needs xxunk the and telly the on football no is there and night stormy and dark a 's it xxmaj . home at are though you xxmaj ? law in mother the visiting is wife the xxmaj ? over - sleep a on away are xxunk your so xxmaj
1,"and , fish some and , xxunk and xxunk only but , animal of kind each of something see we xxmaj . is it xxunk or pole north from far how indication an with starts "" chapter "" each xxmaj . parts these of one than more in appear , xxunk and elephant , bear polar , families animal 3 . xxunk xxmaj to pole xxmaj north xxmaj from earth on life","some and , fish some and , xxunk and xxunk only but , animal of kind each of something see we xxmaj . is it xxunk or pole north from far how indication an with starts "" chapter "" each xxmaj . parts these of one than more in appear , xxunk and elephant , bear polar , families animal 3 . xxunk xxmaj to pole xxmaj north xxmaj from earth on"
2,"boys of team a on own her hold to try frankie xxmaj watch to painful quite was it , myself player baseball a being xxmaj . baseball and ballet : xxunk two her and , grandmother her , friend best her between balance a keep to hard work to has she how see to interesting 's it xxmaj . off … bit a is frankie xxmaj and hazel xxmaj between relationship the",", boys of team a on own her hold to try frankie xxmaj watch to painful quite was it , myself player baseball a being xxmaj . baseball and ballet : xxunk two her and , grandmother her , friend best her between balance a keep to hard work to has she how see to interesting 's it xxmaj . off … bit a is frankie xxmaj and hazel xxmaj between relationship"
3,"the to death out deals xxunk he as xxunk appropriately is "" xxunk xxmaj the "" of delon xxmaj xxunk xxmaj . killings sudden often and savage of lots with entertainment solid as qualifies still it but , predictable rather is "" guns xxmaj big "" , indeed xxmaj . wire the to down right mob the for gunman career a about xxunk this take "" death xxmaj of ring "" of","heads the to death out deals xxunk he as xxunk appropriately is "" xxunk xxmaj the "" of delon xxmaj xxunk xxmaj . killings sudden often and savage of lots with entertainment solid as qualifies still it but , predictable rather is "" guns xxmaj big "" , indeed xxmaj . wire the to down right mob the for gunman career a about xxunk this take "" death xxmaj of ring """
4,"oates xxmaj warren xxmaj great the wasting completely 's it , crime real 's film the know to want you if but xxmaj . economically more much all it says , xxunk xxmaj the across xxmaj , song title beautiful 's xxunk xxmaj xxunk xxmaj unfortunately xxmaj . jarring more the all is melodrama into xxunk sudden the that xxunk and flat so is film the of rest the that is clumsy","in oates xxmaj warren xxmaj great the wasting completely 's it , crime real 's film the know to want you if but xxmaj . economically more much all it says , xxunk xxmaj the across xxmaj , song title beautiful 's xxunk xxmaj xxunk xxmaj unfortunately xxmaj . jarring more the all is melodrama into xxunk sudden the that xxunk and flat so is film the of rest the that is"
5,"xxmaj tom xxmaj with big xxmaj loved i \n\n . good is which … happen n't did that , wow but … appear to mother god fairy the for waiting was xxmaj … corny so xxup * xxunk * … part past the to back going the … but , okay was it thought i end the till up xxmaj … once than more done been has it … but , movie","hanks xxmaj tom xxmaj with big xxmaj loved i \n\n . good is which … happen n't did that , wow but … appear to mother god fairy the for waiting was xxmaj

So how do we know? Let's look at the first batch in our validation sets:

In [ ]:
b_f = next(iter(dls[1]))
b_b = next(iter(dls_r[1]))

In [ ]:
b_f[0]

LMTensorText([[   2,    8,   28,  ...,    0,   12,  211],
        [ 637,   31,   33,  ...,   41,  286,   18],
        [  15,  345, 3432,  ...,  114, 2502,   12],
        ...,
        [ 246,    0,   11,  ...,   19,   85,   44],
        [ 500,   37,   43,  ..., 3801,  798,   66],
        [3014,   10,    8,  ...,   24,  124,   54]], device='cuda:0')

In [ ]:
b_b[0]

LMTensorText([[ 211,   12,    0,  ...,   28,    8,    2],
        [  18,  286,   41,  ...,   33,   31,  637],
        [  12, 2502,  114,  ..., 3432,  345,   15],
        ...,
        [  44,   85,   19,  ...,   11,    0,  246],
        [  66,  798, 3801,  ...,   43,   37,  500],
        [  54,  124,   24,  ...,    8,   10, 3014]], device='cuda:0')

We can see that each word is simply 100% reversed. There are some other augmentation techniques too, such as translation that we may look at later

The last thing we need is a way to enable us to grab the backwards language model inside of `language_model_learner`:

In [ ]:
def _get_text_vocab(dls):
    vocab = dls.vocab
    if isinstance(vocab, L): vocab = vocab[0]
    return vocab

from fastai2.text.models.core import _model_meta

def language_model_learner(dls, arch, config=None, drop_mult=1., backwards=True, pretrained=True, pretrained_fnames=None, **kwargs):
    "Create a `Learner` with a language model from `dls` and `arch`."
    vocab = _get_text_vocab(dls)
    model = get_language_model(arch, len(vocab), config=config, drop_mult=drop_mult)
    meta = _model_meta[arch]
    learn = LMLearner(dls, model, loss_func=CrossEntropyLossFlat(), splitter=meta['split_lm'], **kwargs)
    #TODO: add backard
    url = 'url_bwd' if backwards else 'url'
    if pretrained or pretrained_fnames:
        if pretrained_fnames is not None:
            fnames = [learn.path/learn.model_dir/f'{fn}.{ext}' for fn,ext in zip(pretrained_fnames, ['pth', 'pkl'])]
        else:
            if url not in meta:
                warn("There are no pretrained weights for that architecture yet!")
                return learn
            model_path = untar_data(meta[url] , c_key='model')
            fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
        learn = learn.load_pretrained(*fnames)
    return learn

Now this is still in the in-progress stage at this moment, it'll be more in-house once it's finished and I'll update this notebook accordingly

## Training

So for training, this will be an ensemble of two models, a forwards and a backwards model. We'll train all the models all at once and save them away into an array:

In [ ]:
def train_models(models:list):
    names = ['fwd', 'bwd']
    for i, model in enumerate(models):
        lr = 1e-2
        lr *= model.dls.bs/48
        model = model.to_fp16()
        model.fine_tune(5)
        model.save_encoder(f'{names[i]}_fine_tuned_enc')
    return models

In [ ]:
fwd = language_model_learner(dls, AWD_LSTM, backwards=False, metrics=[accuracy, Perplexity()])
bwd = language_model_learner(dls_r, AWD_LSTM, backwards=True, metrics=[accuracy, Perplexity()])

In [ ]:
spp = train_models([fwd, bwd])

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.860167,4.186048,0.269258,65.762352,00:07


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.682951,4.120397,0.273857,61.583664,00:09
1,4.579101,4.061773,0.277470,58.077179,00:09
2,4.482619,4.028481,0.280158,56.175491,00:09
3,4.421196,4.012850,0.279926,55.304253,00:09
4,4.387582,4.009764,0.279356,55.133842,00:09


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,7.599288,6.916414,0.058263,1008.696106,00:08


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.295796,5.560562,0.113570,259.968933,00:09
1,5.157556,4.005322,0.291526,54.889469,00:09
2,4.162626,3.041760,0.433004,20.942068,00:09
3,3.536577,2.673681,0.494720,14.493217,00:09
4,3.250588,2.616126,0.504535,13.682611,00:09


Now we just repeat what we did last time!

In [ ]:
blocks = (TextBlock.from_df('text', res_col_name='tok_text', seq_len=fwd.dls.seq_len,
                            vocab=fwd.dls.vocab), CategoryBlock())

imdb_class = DataBlock(blocks=blocks,
                       get_x=ColReader('tok_text'),
                       get_y=ColReader('label'),
                       splitter=ColSplitter(col='is_valid'))
fwd_dls = imdb_class.dataloaders(df, bs=32)

In [ ]:
def reverse_text(x:TensorText): return torch.stack([a.flip(0) for a in x])

In [ ]:
blocks = (TextBlock.from_df('text', res_col_name='tok_text', seq_len=bwd.dls.seq_len,
                            vocab=bwd.dls.vocab), CategoryBlock())

imdb_class = DataBlock(blocks=blocks,
                       get_x=ColReader('tok_text'),
                       get_y=ColReader('label'),
                       splitter=ColSplitter(col='is_valid'),
                       batch_tfms=reverse_text)
bwd_dls = imdb_class.dataloaders(df, bs=32)

In [ ]:
bwd_dls.show_batch()

,text,category
0,". xxunk of bowl a into hands my stick go to going 'm i xxmaj , me excuse 'll you if so xxmaj ? him about film a like i can how ultimately so , him like n't do i way either xxmaj ' ? xxunk - xxunk xxunk my into get to want xxunk , xxunk - bay hey ` , car his from girls to shouting up end just 'll who , latino xxmaj blooded - hot stereotypical a merely is vargas xxmaj victor xxmaj , film the by xxunk further any without because 's it maybe xxmaj . xxunk was i when xxunk xxmaj xxunk xxmaj my with playing was i while laid getting were who guys those of me reminds vargas xxmaj victor xxmaj because 's it maybe xxmaj . film the for feelings my xxunk vargas xxmaj victor xxmaj for respect of lack my ,",negative
1,". could possibly very we that feeling it from away come we that is film this about thing best the ; film this in depicted romance the have to as lucky so be all should we xxmaj . role his in xxunk is stewart xxmaj and , charming , funny , sweet 's it xxmaj . instead christmas xxmaj this corner xxup the xxup around xxup shop xxup the xxup watching in done , good of lot a fact in and , harm no be 'd there but , film other that of merits the from xxunk to not xxmaj . life xxup wonderful xxup a 's it xxup about talk they when on out missing are people that film christmas xxmaj stewart xxmaj jimmy xxmaj the really is * this * \n\n ) . mail xxup got xxup 've you xxup like remake a in even not , no",positive
2,") 10 / xxunk the about xxunk a just even know who all of hearts the break to bound is it xxmaj . costs all at avoided be to , film soderbergh xxmaj a for , xxunk -- misery irritating , confusing just is xxunk cinema xxunk this of rest the xxmaj \n\n . subject 's film the of execution and capture -- tragic yet -- dramatic the and , humour aforementioned the by redeemed only is half second entire the xxmaj . xxunk and misery of picture a it painting , xxunk xxmaj for xxunk no certainly is part2 xxmaj aka xxunk xxmaj the : che \n\n . "" turgid and slow unbearably almost feels … "" which , "" detail excruciating in xxunk xxmaj in campaign final 's che xxmaj xxunk "" , there from "" downhill rapidly goes "" actually part2 xxmaj , "" uneven "" was",negative
3,". end xxup the xxup . other each at look they xxmaj . window the down xxunk , up drives movie the in earlier her pay n't did who contractor the xxmaj . business does normally she where corner the at xxunk and , street the down walking prostitute the see we and , off goes gun the , eventually xxmaj . aim to where her telling , mouth his inside gun the holding , hand her holding , insists he but , it do to want not does prostitute the xxmaj . it do n't could but , himself kill to tried he xxmaj . him kill will she if cash in 0 3 xxrep , xxunk $ him give to wants she xxmaj . xxunk xxmaj la xxmaj out started who prostitute same the to , beginning the to back come we and , xxunk brooklyn xxmaj the",positive
4,". there xxunk interesting have they least at , xxunk an for office 's doctor a at waiting fun more have 'll you xxmaj . else something do go = boring xxmaj + boring xxmaj + boring xxmaj : points three my summarize to xxmaj . xxunk mind is handled was show this which in manner incompetent xxunk the but , nature this of show a up putting first for bravo xxmaj commend do i . xxunk be to needed that issues were these felt i but winded long was this if sorry xxmaj \n\n . compromise to refuses show this xxmaj . kisses xxunk and xxunk xxunk than more want na gon are shows tv xxup reality watch who people xxmaj . on * 4 xxrep the come but , men homosexual of nature xxunk xxunk the up play to wanting not them understand can i . ridiculous 's",negative
5,"10 / 8 . games 3d of fans all to this recommended i . appeared first he after years fifteen nearly ) ? especially even maybe xxunk , recognition the deserves he … shoes 's . xxunk into step and bunker the enter to door the open , xxunk xxmaj the up load so xxmaj . one this to existence their owe , genre the of rest 

In [ ]:
fwd = text_classifier_learner(fwd_dls, AWD_LSTM, backwards=False, metrics=[accuracy])
bwd = text_classifier_learner(bwd_dls, AWD_LSTM, backwards=True, metrics=[accuracy])

In [ ]:
fwd.load_encoder('fwd_fine_tuned_enc');
bwd.load_encoder('bwd_fine_tuned_enc');

In [ ]:
def train_class(models:list):
    for i, model in enumerate(models):
        lr = 0.04365158379077912
        adj = 2.6**4
        model.fit_one_cycle(1, lr)
        model.freeze_to(-2)
        model.fit_one_cycle(1, slice(lr/adj, lr))
        model.freeze_to(-3)
        lr /= 2
        model.fit_one_cycle(1, slice(lr/adj, lr))
        model.unfreeze()
        lr /= 5
        model.fit_one_cycle(2, slice(lr/adj, lr))
    return models

In [ ]:
models = [fwd,bwd]

In [ ]:
models = train_class(models)

epoch,train_loss,valid_loss,accuracy,time
0,0.655381,0.621292,0.620000,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.628992,0.533796,0.735000,00:05


epoch,train_loss,valid_loss,accuracy,time
0,0.416536,0.434743,0.800000,00:07


epoch,train_loss,valid_loss,accuracy,time
0,0.365156,0.416463,0.805000,00:09
1,0.282378,0.416023,0.810000,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.759698,0.584659,0.705000,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.695663,0.549426,0.735000,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.541979,0.501881,0.780000,00:07


epoch,train_loss,valid_loss,accuracy,time
0,0.417659,0.531670,0.785000,00:09
1,0.333225,0.554037,0.795000,00:09


Now that we have our model, let's ensemble!

In [ ]:
fwd_p, y = models[0].get_preds()
bwd_p, _ = models[1].get_preds()

In [ ]:
p = (fwd_p + bwd_p) / 2

In [ ]:
accuracy(p, y)

tensor(0.8300)

Let's compare that to both our others:

In [ ]:
accuracy(fwd_p, y)

tensor(0.8100)

In [ ]:
accuracy(bwd_p, y)

tensor(0.7950)

|     Model Type    | Accuracy |
|:-----------------:|:--------:|
| Forwards (Normal) |  81.00%  |
|     Backwards     |  79.50%  |
|      Ensemble     |  83.00%  |
|<img width=200/>|<img width=200/>|

We can see that even though they were well below 83%, combined they boosted up quite high!